## Netztransparenz API

get data from a website of the four big TSO companies in germany

Website:
https://extranet.netztransparenz.de/DesktopModules/LotesDataManagementExtranet/Swagger/index.html?version=public#/Data/get_api_v1_data_redispatch__dateFrom___dateTo_

In [24]:
# import necessary packages
import pandas as pd
import requests
import sys
import os
import io

In [19]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

# Path to the token file in your Google Drive
token_file_path = '/content/drive/MyDrive/ignore/netztransparenz_token.txt'

# Read the token from the file
with open(token_file_path, 'r') as file:
    netztransparenz_token = file.read().strip()

os.environ['IPNT_CLIENT_ID'] = 'cm_app_ntp_id_f38e0b3b390b40e9aff5331846dc1d58'
os.environ['IPNT_CLIENT_SECRET'] = netztransparenz_token

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
IPNT_CLIENT_ID = os.environ.get('IPNT_CLIENT_ID')
IPNT_CLIENT_SECRET = os.environ.get('IPNT_CLIENT_SECRET')

ACCESS_TOKEN_URL = "https://identity.netztransparenz.de/users/connect/token"

# Ask for the token providing above authorization data
response = requests.post(ACCESS_TOKEN_URL,
                data = {
                        'grant_type': 'client_credentials',
                        'client_id': IPNT_CLIENT_ID,
                        'client_secret': IPNT_CLIENT_SECRET
        })

# Parse the token from the response if the response was OK
if response.ok:
    TOKEN = response.json()['access_token']
else:
    print(f'Error retrieving token\n{response.status_code}:{response.reason}',
        file = sys.stderr)
    exit(-1)

# Provide URL to request health info on API
myURL = "https://ds.netztransparenz.de/api/v1/health"
response = requests.get(myURL, headers = {'Authorization': 'Bearer {}'.format(TOKEN)})
print(response.text, file = sys.stdout)

"OK"


## Redispatch data

In [29]:
url = 'https://ds.netztransparenz.de/api/v1/data/redispatch/2020-01-01T00:00:00/2024-01-11T00:00:00'
#url = 'https://ds.netztransparenz.de/api/v1/data/redispatch'
response = requests.get(url, headers = {'Authorization': 'Bearer {}'.format(TOKEN)})

data_redispatch = response.text
df_redispatch = pd.read_csv(io.StringIO(data_redispatch), sep=';')
df_redispatch = df_redispatch[df_redispatch['RICHTUNG'] == 'Wirkleistungseinspeisung reduzieren']

df_redispatch_tennet = df_redispatch[df_redispatch['NETZREGION'] == 'RZ TenneT DE']

In [30]:
df_redispatch_tennet.head()

,BEGINN_DATUM,BEGINN_UHRZEIT,ZEITZONE_VON,ENDE_DATUM,ENDE_UHRZEIT,ZEITZONE_BIS,NETZREGION,GRUND_DER_MASSNAHME,RICHTUNG,MITTLERE_LEISTUNG_MW,MAXIMALE_LEISTUNG_MW,GESAMTE_ARBEIT_MWH,ANWEISENDER_UENB,ANFORDERNDER_UENB,BETROFFENE_ANLAGE
1,31.12.2019,23:00,UTC,01.01.2020,17:00,UTC,RZ TenneT DE,Spannungsbedingter Redispatch,Wirkleistungseinspeisung reduzieren,230,230,4140,TenneT DE,TenneT DE,Börse
7,02.01.2020,15:00,UTC,02.01.2020,23:00,UTC,RZ TenneT DE,Strombedingter Redispatch,Wirkleistungseinspeisung reduzieren,98,100,788,TenneT DE,TenneT DE,Grohnde
10,02.01.2020,17:00,UTC,02.01.2020,23:00,UTC,RZ TenneT DE,Strombedingter Redispatch,Wirkleistungseinspeisung reduzieren,98,100,588,TenneT DE,TenneT DE,Grohnde
16,02.01.2020,23:00,UTC,03.01.2020,03:00,UTC,RZ TenneT DE,Strombedingter Redispatch,Wirkleistungseinspeisung reduzieren,175,200,700,TenneT DE,TenneT DE,Grohnde
21,03.01.2020,04:00,UTC,03.01.2020,10:00,UTC,RZ TenneT DE,Strombedingter Redispatch,Wirkleistungseinspeisung reduzieren,50,50,300,TenneT DE,TenneT DE,Grohnde


In [28]:
df_redispatch_tennet['BETROFFENE_ANLAGE'].unique()

array(['Staudinger 5', 'Börse', 'Grohnde',
       'Rheinhafen-Dampfkraftwerk Karlsruhe Block 8', 'Zolling 5',
       'Boxberg,Schwarze Pumpe', 'Lippendorf (EnBW)',
       'Lippendorf (LEAG),Lippendorf (EnBW)',
       'Boxberg,Jänschwalde,Schwarze Pumpe', 'Brokdorf',
       'Heizkraftwerk Altbach/Deizisau Block 2', 'Waldeck 5', 'Scholven',
       'Walsum', 'Weisweiler', 'Herne', 'Völklingen HKV',
       'Lippendorf (LEAG),Schwarze Pumpe', 'Vorarlberger Illwerke AG',
       'Grosskraftwerk Mannheim AG', 'Isar 2 (SWM)', 'Erzhausen 2',
       'Lippendorf (LEAG)', 'SW München KW-Pool', 'Bremen 6', 'Farge',
       'Moorburg', 'Rheinhafen-Dampfkraftwerk Karlsruhe Block 7',
       'Emsland', 'Boxberg,Jänschwalde,Lippendorf (LEAG),Schwarze Pumpe',
       'Jänschwalde,Schwarze Pumpe', 'Boxberg,Jänschwalde',
       'Reservekraftwerk Staudinger 4',
       'Boxberg,Jänschwalde,Rostock,Schwarze Pumpe',
       'Boxberg,Lippendorf (LEAG),Schwarze Pumpe', 'Schwarze Pumpe',
       'Bremen 15', 'Kraftwer

## Spotmarket

In [ ]:
url = 'https://ds.netztransparenz.de/api/v1/data/Spotmarktpreise'

response = requests.get(url, headers = {'Authorization': 'Bearer {}'.format(TOKEN)})

data_spotmarket= response.text
df_spotmarket = pd.read_csv(io.StringIO(data_spotmarket), sep=';')

In [ ]:
df_spotmarket.head()

,Datum,von,Zeitzone von,bis,Zeitzone bis,Spotmarktpreis in ct/kWh
0,31.12.2020,23:00,UTC,00:00,UTC,"5,087"
1,01.01.2021,00:00,UTC,01:00,UTC,"4,819"
2,01.01.2021,01:00,UTC,02:00,UTC,"4,468"
3,01.01.2021,02:00,UTC,03:00,UTC,"4,292"
4,01.01.2021,03:00,UTC,04:00,UTC,"4,039"


## Wind forecast

In [ ]:
url = 'https://ds.netztransparenz.de/api/v1/data/prognose/wind'

response = requests.get(url, headers = {'Authorization': 'Bearer {}'.format(TOKEN)})

data_wind_forecast = response.text
df_wind_forecast = pd.read_csv(io.StringIO(data_wind_forecast), sep=';')

In [ ]:
df_wind_forecast.tail()

,Datum,von,Zeitzone von,bis,Zeitzone bis,50Hertz (MW),Amprion (MW),TenneT TSO (MW),TransnetBW (MW)
451495,2024-02-14,21:45,UTC,22:00,UTC,N.A.,N.A.,N.A.,N.A.
451496,2024-02-14,22:00,UTC,22:15,UTC,N.A.,N.A.,N.A.,N.A.
451497,2024-02-14,22:15,UTC,22:30,UTC,N.A.,N.A.,N.A.,N.A.
451498,2024-02-14,22:30,UTC,22:45,UTC,N.A.,N.A.,N.A.,N.A.
451499,2024-02-14,22:45,UTC,23:00,UTC,N.A.,N.A.,N.A.,N.A.
